In [48]:
import requests
import pandas as pd
import time

API_KEY_FRESHWORK = **************
DOMAIN_FRESHWORK = 'institutadios'
BASE_URL = f'https://{DOMAIN_FRESHWORK}.myfreshworks.com/crm/sales/'

def get_existing_contacts(api_key, domain):
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Token token={api_key}'
    }
    all_contacts = {}
    page = 1
    last_contact_id = None

    while True:
        params = {'per_page': 100, 'page': page}
        if last_contact_id:
            params['last_contact_id'] = last_contact_id
        url = f"{BASE_URL}api/contacts/view/31004512042?per_page=100&sort=created_at"

        response = requests.get(url, headers=headers, params=params)
        if not response.ok:
            print(f"Erreur lors de la récupération des données : {response.status_code}")
            return

        response_data = response.json()
        if 'contacts' not in response_data:
            print("Erreur dans la structure de la réponse JSON")
            break

        contacts_data = response_data['contacts']
        if not contacts_data:
            break

        for contact in contacts_data:
            bitrix_id = contact['custom_field'].get('cf_id_bitrix')
            if bitrix_id:
                all_contacts[bitrix_id] = contact

        last_contact_id = contacts_data[-1]['id']
        page += 1
        time.sleep(1)

    print(f"Contacts récupérés avec succès : {len(all_contacts)}")
    return all_contacts

def main():
    existing_contacts = get_existing_contacts(API_KEY_FRESHWORK, DOMAIN_FRESHWORK)
    data = {
        "internal_id": [],  
        "first_name": [],
        "last_name": [],
        "email": [],
        "mobile_number": [],
        "updated_at": []
    }
    custom_field_data = {}
    
    custom_fields = set()
    for contact in existing_contacts.values():
        custom_fields.update(contact.get("custom_field", {}).keys())
    
    for field in custom_fields:
        custom_field_data[field] = []
    for contact in existing_contacts.values():
        # Ajouter l'ID interne
        data["internal_id"].append(contact.get("id", ""))

        data["first_name"].append(contact.get("first_name", ""))
        data["last_name"].append(contact.get("last_name", ""))
        data["email"].append(contact.get("email", ""))
        data["mobile_number"].append(contact.get("mobile_number", ""))   
        
        custom_field = contact.get("custom_field", {})
        data.setdefault("cf_id_bitrix", []).append(custom_field.get("cf_id_bitrix", ""))
        
        updated_at = contact.get("updated_at", "")
        if updated_at:
            updated_at = updated_at.split("T")[0]
        data["updated_at"].append(updated_at)

        # Ajout des autres champs de custom_field
        for field in custom_fields:
            custom_field_data[field].append(custom_field.get(field, ""))

    df_main = pd.DataFrame(data)
    df_custom = pd.DataFrame(custom_field_data)
    df_combined = pd.concat([df_main, df_custom], axis=1)
    
    df_combined["updated_at"] = pd.to_datetime(df_combined["updated_at"])
    df_combined["month_year"] = df_combined["updated_at"].dt.to_period('M')
    df_combined = df_combined.sort_values(by="month_year", ascending=False)
    df_combined = df_combined.drop(columns=["month_year"])
    df_combined = df_combined.reset_index(drop=True)
    
    df_combined.to_csv('contacts_rs.csv', index=False, encoding='utf-8')
    
    return df_combined

if __name__ == '__main__':
    contacts_df = main()
    print(contacts_df)


Contacts récupérés avec succès : 45395
       internal_id first_name            last_name  \
0      31099102049       None         +33699095985   
1      31043226095       None                 None   
2      31043226092   Bernardo                 None   
3      31043226091       Guay               Louise   
4      31043226089  GROSDOIGT              Olivier   
...            ...        ...                  ...   
45390  31045862776       None  Balasundaram joseph   
45391  31045861921       None  Veronique Rousseau    
45392  31045861705       None    Beltier Laetitia    
45393  31044327693      Cathy              GUERREO   
45394  31043045756    Cherine                KEMEL   

                                 email mobile_number updated_at cf_id_bitrix  \
0                                 None  +33699095985 2024-09-27  33699095985   
1            philippeelain@laposte.net     679176384 2024-09-26        79219   
2      bernardo_g_fernanda@hotmail.com   32478447832 2024-09-26        7

In [49]:
import pandas as pd

doublonss = pd.read_csv("contacts_rs.csv")

doublonss["updated_at"] = pd.to_datetime(doublonss["updated_at"])

doublonss["month_year"] = doublonss["updated_at"].dt.to_period('M')

doublonss = doublonss.sort_values(by="month_year", ascending=False)

doublonss = doublonss.drop(columns=["month_year"])

doublonss = doublonss.reset_index(drop=True)

doublonss


C:\Users\PC\AppData\Local\Temp\ipykernel_8928\564244643.py:3: DtypeWarning: Columns (24,27) have mixed types. Specify dtype option on import or set low_memory=False.
  doublonss = pd.read_csv("contacts_rs.csv")


,internal_id,first_name,last_name,email,mobile_number,updated_at,cf_id_bitrix,cf_webinar_date,cf_replay_link,cf_date_de_naissance,...,cf_url_de_la_source,cf_objectif_formul,cf_webinar_time,cf_canal_dacquisition,cf_date_de_creation,cf_type_de_prospect,cf_evenement_dacquisition,cf_note2,cf_message_publicitaire_utm_content,cf_live_room_link
0,31099102049,NaN,+33699095985,NaN,+33699095985,2024-09-27,33699095985,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,31054830674,Nathalie Antunes,NaN,nathalieantunes@wanadoo.fr,664225148,2024-09-24,113049,NaN,NaN,NaN,...,https://event.webinarjam.com/channel/mastercla...,Auto webinar jeudi,NaN,NaN,"Thu, 9 Nov 2023, 10:15 AM",NaN,NaN,NaN,NaN,NaN
2,31054830577,Marie-Claude,NaN,marie-claude.des@videotron.ca,+5149794899,2024-09-23,112963+111225,NaN,NaN,NaN,...,https://institutadios.easywebinar.live/session...,NaN,NaN,NaN,2023-09-21 21:39:04,NaN,NaN,NaN,NaN,NaN
3,31054830588,Dalila,Safer,saferdalila@gmail.com,+44616762398,2024-09-23,112977+111291,NaN,NaN,NaN,...,https://institutadios.easywebinar.live/session...,NaN,NaN,NaN,2023-09-22 20:58:13,NaN,NaN,NaN,NaN,NaN
4,31054830680,Christine Hauri,NaN,chris.hauri@gmail.com,796063642,2024-09-24,113059,NaN,NaN,NaN,...,https://event.webinarjam.com/channel/mastercla...,Auto webinar jeudi,NaN,NaN,"Thu, 9 Nov 2023, 07:03 AM",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45390,31045865836,NaN,Vanessa tessier,NaN,0769288942,2023-12-21,0769288942,NaN,NaN,NaN,...,NaN,Alcool,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45391,31045187749,NaN,SANS NOM,NaN,0680734405,2023-12-15,06 80 73 44 05,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45392,31045499386,NaN,SANS NOM,NaN,0660736095,2023-12-19,0660736095,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45393,31046428722,NaN,SANS NOM,NaN,NaN,2023-12-28,0779229688,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
del doublonss["cf_evenement_dacquisition"]
del doublonss["cf_budget_journalier_utm_medium"]
del doublonss["cf_webinar_time"]
del doublonss["cf_webinar_date"]
del doublonss["cf_a_assiste_au_live"]
del doublonss["cf_inscription_utm_source"]
del doublonss["cf_type_de_prospect"]
del doublonss["cf_message_publicitaire_utm_content"]
del doublonss["cf_nom_de_la_campagne_de_publicit_utm_campaign"]
del doublonss["cf_note2"]
del doublonss["cf_url_de_la_source"]
del doublonss["cf_live_room_link"]
del doublonss["cf_note1"]
del doublonss["cf_date_de_naissance"]
del doublonss["cf_canal_dacquisition"]
del doublonss["cf_replay_link"]
del doublonss["cf_time_zone"]
del doublonss["cf_ciblage_utm_term"]
del doublonss["cf_action_initiale_du_prospect"]
del doublonss["cf_nom_du_prospect"]
del doublonss["cf_objectif_formul"]
del doublonss["cf_id_bitrix"]

doublonss.to_csv("Listes_des_doublons_fresh_email.csv", index=False)
doublonss

,internal_id,first_name,last_name,email,mobile_number,updated_at,cf_datecre_bitrix,cf_id_bitrix.1,cf_date_de_creation
0,31099102049,NaN,+33699095985,NaN,+33699095985,2024-09-27,NaN,33699095985,NaN
1,31054830674,Nathalie Antunes,NaN,nathalieantunes@wanadoo.fr,664225148,2024-09-24,NaN,113049,"Thu, 9 Nov 2023, 10:15 AM"
2,31054830577,Marie-Claude,NaN,marie-claude.des@videotron.ca,+5149794899,2024-09-23,NaN,112963+111225,2023-09-21 21:39:04
3,31054830588,Dalila,Safer,saferdalila@gmail.com,+44616762398,2024-09-23,NaN,112977+111291,2023-09-22 20:58:13
4,31054830680,Christine Hauri,NaN,chris.hauri@gmail.com,796063642,2024-09-24,NaN,113059,"Thu, 9 Nov 2023, 07:03 AM"
...,...,...,...,...,...,...,...,...,...
45390,31045865836,NaN,Vanessa tessier,NaN,0769288942,2023-12-21,NaN,0769288942,NaN
45391,31045187749,NaN,SANS NOM,NaN,0680734405,2023-12-15,NaN,06 80 73 44 05,NaN
45392,31045499386,NaN,SANS NOM,NaN,0660736095,2023-12-19,NaN,0660736095,NaN
45393,31046428722,NaN,SANS NOM,NaN,NaN,2023-12-28,NaN,0779229688,NaN


In [51]:
#doublons par email

import pandas as pd

df_bitrixs = pd.read_csv("Listes_des_doublons_fresh_email.csv", encoding='utf-8', low_memory=False)

df_filtre = df_bitrixs.dropna(subset=['email'])

doublons = df_filtre[df_filtre.duplicated(subset=['email'], keep=False)]

doublons = doublons.sort_values(by='email')

if doublons.empty:
    print("Aucun doublon trouvé basé sur les colonnes spécifiées.")
else:
    print(doublons)


       internal_id  first_name        last_name  \
4576   31099082196     Armelle          Casalta   
3908   31085965012     Armelle         Casalta    
4428   31099083221  Anne-Laure          Albouze   
712    31095016244  Anne-Laure          Albouze   
4039   31085964895         NaN  Alice Bournival   
...            ...         ...              ...   
4401   31099082182   Stéphanie         Chareyre   
22474  31043235805      Sylvie           Pachis   
4298   31099082152      sylvie           pachis   
4478   31099082107   Veronique           MARTIN   
40695  31043256477   Véronique          MARTIN    

                                 email mobile_number  updated_at  \
4576             a.casalta@laposte.net   33611477782  2024-09-27   
3908             a.casalta@laposte.net           NaN  2024-09-22   
4428                al.albouze@free.fr   33612071797  2024-09-27   
712                 al.albouze@free.fr  +33612071797  2024-09-22   
4039        alice.bournival@icloud.com         

In [53]:

del doublons["cf_datecre_bitrix"]
del doublons["cf_id_bitrix.1"]
doublons.to_csv("Modif_Listes_des_doublons_fresh_email.csv", index=False)
doublons

,internal_id,first_name,last_name,email,mobile_number,updated_at,cf_date_de_creation
4576,31099082196,Armelle,Casalta,a.casalta@laposte.net,33611477782,2024-09-27,"Mon, 23 Sep 2024, 12:55 AM"
3908,31085965012,Armelle,Casalta,a.casalta@laposte.net,NaN,2024-09-22,"2024-07-01 21:52:37 + Thu, 15 Aug 2024, 09:46 ..."
4428,31099083221,Anne-Laure,Albouze,al.albouze@free.fr,33612071797,2024-09-27,"Tue, 24 Sep 2024, 01:30 PM"
712,31095016244,Anne-Laure,Albouze,al.albouze@free.fr,+33612071797,2024-09-22,"Fri, 16 Aug 2024, 06:02 AM"
4039,31085964895,NaN,Alice Bournival,alice.bournival@icloud.com,NaN,2024-09-22,"2024-07-03 11:06:40 + Wed, 14 Aug 2024, 12:53 AM"
...,...,...,...,...,...,...,...
4401,31099082182,Stéphanie,Chareyre,stephanie_chareyre@yahoo.fr,33667087504,2024-09-27,"Mon, 23 Sep 2024, 11:53 AM"
22474,31043235805,Sylvie,Pachis,sylviepachis@gmail.com,+33637262466,2024-09-23,"Saturday, September 30, 2023"
4298,31099082152,sylvie,pachis,sylviepachis@gmail.com,33637262466,2024-09-27,"Mon, 23 Sep 2024, 09:53 PM"
4478,31099082107,Veronique,MARTIN,veronique.martin555@hotmail.com,262692666681,2024-09-27,"Tue, 24 Sep 2024, 09:47 AM"


In [54]:
import gspread
import csv
from datetime import datetime


gc = gspread.service_account(filename="anomalieV2.json")

sh = gc.open_by_key('1cMEvZRMf8HWIeWIuZfGCojR4oMzPaJFGjnIEb4_4q8g')

worksheet = sh.worksheet("Anomalie 1-Doublons (email)")

worksheet.clear()

with open("Modif_Listes_des_doublons_fresh_email.csv", 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    rows = list(reader)

last_col_num = ord('A') + len(rows[0]) - 1
last_col_letter = chr(last_col_num) if last_col_num <= ord('Z') else 'Z'
cell_range = f'A2:{last_col_letter}{len(rows)+1}'
cell_list = worksheet.range(cell_range)
for cell, value in zip(cell_list, [cell for row in rows for cell in row]):
    cell.value = value

worksheet.update_cells(cell_list)

now = datetime.now()
date_string = now.strftime("%d/%m/%Y à %H:%M:%S")
worksheet.update_acell('J2', date_string)

worksheet = None


In [58]:
#doublons par mobile

import pandas as pd

df_bitrixs = pd.read_csv("Listes_des_doublons_fresh_email.csv", encoding='utf-8', low_memory=False)

df_filtred = df_bitrixs.dropna(subset=['mobile_number'])

doublonses = df_filtred[df_filtred.duplicated(subset=['mobile_number'], keep=False)]

doublonses = doublonses.sort_values(by='mobile_number')

if doublonses.empty:
    print("Aucun doublon trouvé basé sur les colonnes spécifiées.")
else:
    print(doublonses)


       internal_id                 first_name                  last_name  \
8890   31055018663                     Hélène                    RAPETTI   
2828   31099082095                     Hélène                    Rapetti   
653    31095017034                  Christine                  Fonteneau   
2858   31099082087                  Christine                  Fonteneau   
2970   31095017113                  Claudette                   Dekeyser   
...            ...                        ...                        ...   
28740  31043201853  Laurence Baptiste-Weppler  Laurence Baptiste-Weppler   
17808  31053675550                    Chantal                      TILMA   
34091  31043179895                    Chantal                      Tilma   
10608  31053853231               Marie-Astrid                     Seguin   
30184  31043172248               Marie-Astrid                     Seguin   

                                 email mobile_number  updated_at  \
8890              r

In [59]:
doublonses

,internal_id,first_name,last_name,email,mobile_number,updated_at,cf_datecre_bitrix,cf_id_bitrix.1,cf_date_de_creation
8890,31055018663,Hélène,RAPETTI,rapettihelene@sfr.fr,+33626639352,2024-09-27,02/11/2022 18:05:27,84059+35701,26/09/2020 18:53:59
2828,31099082095,Hélène,Rapetti,rapettihelene06@sfr.fr,+33626639352,2024-09-27,NaN,93.21.67.132,"Tue, 24 Sep 2024, 01:40 PM"
653,31095017034,Christine,Fonteneau,fonteneau.christine@akeonet.com,+33650285249,2024-09-27,NaN,79.95.86.243,"Thu, 22 Aug 2024, 05:41 PM"
2858,31099082087,Christine,Fonteneau,fonteneau.christine@akeonet.com,+33650285249,2024-09-27,NaN,77.152.222.33,"Tue, 24 Sep 2024, 08:36 PM"
2970,31095017113,Claudette,Dekeyser,clmallet@yahoo.fr,+33663201708,2024-09-27,NaN,77.133.249.188,"Thu, 22 Aug 2024, 06:13 AM"
...,...,...,...,...,...,...,...,...,...
28740,31043201853,Laurence Baptiste-Weppler,Laurence Baptiste-Weppler,laurenceweppler@hotmail.com,695013177,2024-09-24,06/04/2022 08:18:14,63759,2022-01-26 10:55:50
17808,31053675550,Chantal,TILMA,NaN,71371334,2024-09-24,05/11/2020 00:03:16,20322,05/11/2020
34091,31043179895,Chantal,Tilma,NaN,71371334,2024-09-25,15/12/2021 09:44:21,55681,05/11/2020 00:03:16
10608,31053853231,Marie-Astrid,Seguin,NaN,767973211,2024-09-27,18/03/2021 13:44:16,30352,NaN


In [60]:

del doublonses["cf_datecre_bitrix"]
del doublonses["cf_id_bitrix.1"]
doublonses.to_csv("Modif_listes_des_doublons_fresh_work_number.csv", index=False)
doublonses

,internal_id,first_name,last_name,email,mobile_number,updated_at,cf_date_de_creation
8890,31055018663,Hélène,RAPETTI,rapettihelene@sfr.fr,+33626639352,2024-09-27,26/09/2020 18:53:59
2828,31099082095,Hélène,Rapetti,rapettihelene06@sfr.fr,+33626639352,2024-09-27,"Tue, 24 Sep 2024, 01:40 PM"
653,31095017034,Christine,Fonteneau,fonteneau.christine@akeonet.com,+33650285249,2024-09-27,"Thu, 22 Aug 2024, 05:41 PM"
2858,31099082087,Christine,Fonteneau,fonteneau.christine@akeonet.com,+33650285249,2024-09-27,"Tue, 24 Sep 2024, 08:36 PM"
2970,31095017113,Claudette,Dekeyser,clmallet@yahoo.fr,+33663201708,2024-09-27,"Thu, 22 Aug 2024, 06:13 AM"
...,...,...,...,...,...,...,...
28740,31043201853,Laurence Baptiste-Weppler,Laurence Baptiste-Weppler,laurenceweppler@hotmail.com,695013177,2024-09-24,2022-01-26 10:55:50
17808,31053675550,Chantal,TILMA,NaN,71371334,2024-09-24,05/11/2020
34091,31043179895,Chantal,Tilma,NaN,71371334,2024-09-25,05/11/2020 00:03:16
10608,31053853231,Marie-Astrid,Seguin,NaN,767973211,2024-09-27,NaN


In [61]:
import gspread
import csv
from datetime import datetime

gc = gspread.service_account(filename="anomalieV2.json")

sh = gc.open_by_key('1cMEvZRMf8HWIeWIuZfGCojR4oMzPaJFGjnIEb4_4q8g')

worksheet = sh.worksheet("Anomalie 1-Doublons (mobile_number)")


worksheet.clear() 

with open("Modif_listes_des_doublons_fresh_work_number.csv", 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    rows = list(reader)

last_col_num = ord('A') + len(rows[0]) - 1
last_col_letter = chr(last_col_num) if last_col_num <= ord('Z') else 'Z'
cell_range = f'A2:{last_col_letter}{len(rows)+1}'

cell_list = worksheet.range(cell_range)

for cell, value in zip(cell_list, [cell for row in rows for cell in row]):
    cell.value = value

worksheet.update_cells(cell_list)

now = datetime.now()
date_string = now.strftime("%d/%m/%Y à %H:%M:%S")
worksheet.update_acell('J2', date_string)


worksheet = None


In [62]:
#doublons par autre numéro

import pandas as pd

df_bitrixsp = pd.read_csv("contacts_rs.csv", encoding='utf-8', low_memory=False)

df_filtredp = df_bitrixsp.dropna(subset=['cf_nom_du_prospect'])

doublonsesp = df_filtredp[df_filtredp.duplicated(subset=['cf_nom_du_prospect'], keep=False)]

doublonsesp = doublonsesp.sort_values(by='cf_nom_du_prospect')

if doublonsesp.empty:
    print("Aucun doublon trouvé basé sur les colonnes spécifiées.")
else:
    print(doublonsesp)


       internal_id          first_name       last_name  \
29982  31076548732                 NaN  deham clémence   
29993  31076548735                 NaN  deham clémence   
29061  31076549026                 NaN        plantard   
29791  31076548498                 NaN           dahbi   
28839  31076549568                 NaN          luna 9   
...            ...                 ...             ...   
22744  31067380189         LOGAN JOWEL             NaN   
2140   31043230617  Brigitte Montreuil             NaN   
31077  31077428909                 NaN   ourodjoboawal   
31187  31077428951                 NaN  gerard vaudrey   
2056   31043230453                Eric           Leroy   

                              email mobile_number  updated_at   cf_id_bitrix  \
29982       dehamclemence@gmail.com           NaN  2024-09-23      500012330   
29993      dehamclemence@hotmail.be           NaN  2024-09-23      500012332   
29061             ap37660@gmail.com           NaN  2024-09-23  

In [64]:
del doublonsesp["cf_evenement_dacquisition"]
del doublonsesp["cf_budget_journalier_utm_medium"]
del doublonsesp["cf_webinar_time"]
del doublonsesp["cf_webinar_date"]
del doublonsesp["cf_a_assiste_au_live"]
del doublonsesp["cf_inscription_utm_source"]
del doublonsesp["cf_type_de_prospect"]
del doublonsesp["cf_message_publicitaire_utm_content"]
del doublonsesp["cf_nom_de_la_campagne_de_publicit_utm_campaign"]
del doublonsesp["cf_note2"]
del doublonsesp["cf_url_de_la_source"]
del doublonsesp["cf_live_room_link"]
del doublonsesp["cf_note1"]
del doublonsesp["cf_date_de_naissance"]
del doublonsesp["cf_canal_dacquisition"]
del doublonsesp["cf_replay_link"]
del doublonsesp["cf_time_zone"]
del doublonsesp["cf_ciblage_utm_term"]
del doublonsesp["cf_action_initiale_du_prospect"]
del doublonsesp["cf_objectif_formul"]
del doublonsesp["cf_id_bitrix"]
del doublonsesp["mobile_number"]
doublonsesp

,internal_id,first_name,last_name,email,updated_at,cf_nom_du_prospect,cf_datecre_bitrix,cf_id_bitrix.1,cf_date_de_creation
29982,31076548732,NaN,deham clémence,dehamclemence@gmail.com,2024-09-23,+32493698199,NaN,500012330,2022-11-23 11:55:52
29993,31076548735,NaN,deham clémence,dehamclemence@hotmail.be,2024-09-23,+32493698199,NaN,500012332,2022-11-23 12:33:54
29061,31076549026,NaN,plantard,ap37660@gmail.com,2024-09-23,+33102030405,NaN,500007359,2022-03-07 10:02:57
29791,31076548498,NaN,dahbi,testavec.lea98@gmail.com,2024-09-23,+33102030405,NaN,500007717,2022-04-14 09:05:31
28839,31076549568,NaN,luna 9,clochettewc@gmail.com,2024-09-23,+33102030405,NaN,500019258,2024-01-30 10:46:37
...,...,...,...,...,...,...,...,...,...
22744,31067380189,LOGAN JOWEL,NaN,ydjoellogan96@gmail.com,2024-09-24,97211,NaN,31.37.150.372,11/2/24 18:22
2140,31043230617,Brigitte Montreuil,NaN,montreuilb@cssdlr.gouv.qc.ca,2024-09-26,97809437,27/12/2022 18:06:19,81143,2022-06-08T00:47:52.951Z
31077,31077428909,NaN,ourodjoboawal,ourodjoboawali95@gmail.com,2024-09-24,97809437,NaN,500012766,2022-12-19 21:38:08
31187,31077428951,NaN,gerard vaudrey,gerardvdr633@gmail.com,2024-09-24,984291686,NaN,500012797,2022-12-22 19:10:51


In [65]:

del doublonsesp["cf_datecre_bitrix"]
del doublonsesp["cf_id_bitrix.1"]
doublonsesp

,internal_id,first_name,last_name,email,updated_at,cf_nom_du_prospect,cf_date_de_creation
29982,31076548732,NaN,deham clémence,dehamclemence@gmail.com,2024-09-23,+32493698199,2022-11-23 11:55:52
29993,31076548735,NaN,deham clémence,dehamclemence@hotmail.be,2024-09-23,+32493698199,2022-11-23 12:33:54
29061,31076549026,NaN,plantard,ap37660@gmail.com,2024-09-23,+33102030405,2022-03-07 10:02:57
29791,31076548498,NaN,dahbi,testavec.lea98@gmail.com,2024-09-23,+33102030405,2022-04-14 09:05:31
28839,31076549568,NaN,luna 9,clochettewc@gmail.com,2024-09-23,+33102030405,2024-01-30 10:46:37
...,...,...,...,...,...,...,...
22744,31067380189,LOGAN JOWEL,NaN,ydjoellogan96@gmail.com,2024-09-24,97211,11/2/24 18:22
2140,31043230617,Brigitte Montreuil,NaN,montreuilb@cssdlr.gouv.qc.ca,2024-09-26,97809437,2022-06-08T00:47:52.951Z
31077,31077428909,NaN,ourodjoboawal,ourodjoboawali95@gmail.com,2024-09-24,97809437,2022-12-19 21:38:08
31187,31077428951,NaN,gerard vaudrey,gerardvdr633@gmail.com,2024-09-24,984291686,2022-12-22 19:10:51


In [66]:
doublonsesp["Autres numeros"] = doublonsesp["cf_nom_du_prospect"]
del doublonsesp["cf_nom_du_prospect"]
doublonsesp.to_csv("Modif_Listes_des_doublons_fresh_autre_numeros.csv", index=False)
doublonsesp

,internal_id,first_name,last_name,email,updated_at,cf_date_de_creation,Autres numeros
29982,31076548732,NaN,deham clémence,dehamclemence@gmail.com,2024-09-23,2022-11-23 11:55:52,+32493698199
29993,31076548735,NaN,deham clémence,dehamclemence@hotmail.be,2024-09-23,2022-11-23 12:33:54,+32493698199
29061,31076549026,NaN,plantard,ap37660@gmail.com,2024-09-23,2022-03-07 10:02:57,+33102030405
29791,31076548498,NaN,dahbi,testavec.lea98@gmail.com,2024-09-23,2022-04-14 09:05:31,+33102030405
28839,31076549568,NaN,luna 9,clochettewc@gmail.com,2024-09-23,2024-01-30 10:46:37,+33102030405
...,...,...,...,...,...,...,...
22744,31067380189,LOGAN JOWEL,NaN,ydjoellogan96@gmail.com,2024-09-24,11/2/24 18:22,97211
2140,31043230617,Brigitte Montreuil,NaN,montreuilb@cssdlr.gouv.qc.ca,2024-09-26,2022-06-08T00:47:52.951Z,97809437
31077,31077428909,NaN,ourodjoboawal,ourodjoboawali95@gmail.com,2024-09-24,2022-12-19 21:38:08,97809437
31187,31077428951,NaN,gerard vaudrey,gerardvdr633@gmail.com,2024-09-24,2022-12-22 19:10:51,984291686


In [67]:
import gspread
import csv
from datetime import datetime

gc = gspread.service_account(filename="anomalieV2.json")

sh = gc.open_by_key('1cMEvZRMf8HWIeWIuZfGCojR4oMzPaJFGjnIEb4_4q8g')


worksheet = sh.worksheet("Anomalie 1-Doublons (othernumber)")

worksheet.clear() 

with open("Modif_Listes_des_doublons_fresh_autre_numeros.csv", 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    rows = list(reader)

last_col_num = ord('A') + len(rows[0]) - 1
last_col_letter = chr(last_col_num) if last_col_num <= ord('Z') else 'Z'
cell_range = f'A2:{last_col_letter}{len(rows)+1}'

cell_list = worksheet.range(cell_range)

for cell, value in zip(cell_list, [cell for row in rows for cell in row]):
    cell.value = value

worksheet.update_cells(cell_list)

now = datetime.now()
date_string = now.strftime("%d/%m/%Y à %H:%M:%S")
worksheet.update_acell('J2', date_string)

worksheet = None


In [68]:
# Modèle de mobile identique, mais pas le même format.

import pandas as pd
import re

df_bitrixsv = pd.read_csv("Listes_des_doublons_fresh_email.csv", encoding='utf-8', low_memory=False)

df_filtredd = df_bitrixsv.dropna(subset=['mobile_number'])

def simplify_phone_number(phone):
    return re.sub(r'\D', '', phone)

df_filtredd['simplified_mobile_number'] = df_filtredd['mobile_number'].apply(simplify_phone_number)

doublonsesx = df_filtredd[df_filtredd.duplicated(subset=['simplified_mobile_number'], keep=False)]

doublonsesx = doublonsesx.groupby('simplified_mobile_number').filter(lambda x: len(x['mobile_number'].unique()) > 1)

doublonsesx = doublonsesx.sort_values(by='simplified_mobile_number')

if doublonsesx.empty:
    print("Aucun doublon trouvé basé sur les numéros simplifiés.")
else:
    print(doublonsesx[['mobile_number', 'simplified_mobile_number']])


       mobile_number simplified_mobile_number
41182     0649758683               0649758683
599      +0649758683               0649758683
4292     14503043694              14503043694
901     +14503043694              14503043694
4283     14503941540              14503941540
...              ...                      ...
279     590690643934             590690643934
29568  +596696937550             596696937550
33396   596696937550             596696937550
39688  +596696980678             596696980678
8161    596696980678             596696980678

[588 rows x 2 columns]


C:\Users\PC\AppData\Local\Temp\ipykernel_8928\1567317675.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtredd['simplified_mobile_number'] = df_filtredd['mobile_number'].apply(simplify_phone_number)


In [69]:
doublonsesx

,internal_id,first_name,last_name,email,mobile_number,updated_at,cf_datecre_bitrix,cf_id_bitrix.1,cf_date_de_creation,simplified_mobile_number
41182,31056772427,Beatrice,Cervel,bea.lc85@hotmail.fr,0649758683,2024-05-06,NaN,0649758683,18 août 2021,0649758683
599,31054824717,Béa,tess,formation@institutadios.com,+0649758683,2024-09-26,NaN,111151,2023-09-21T01:32:49+00:00,0649758683
4292,31099082122,Lyne,Gastonguay,lynegastonguay@hotmail.com,14503043694,2024-09-27,NaN,174.93.211.98,"Tue, 24 Sep 2024, 02:38 AM",14503043694
901,31095016429,Lyne,Gastonguay,lynegastonguay@hotmail.com,+14503043694,2024-09-22,NaN,70.24.255.40,"Fri, 16 Aug 2024, 09:02 PM",14503043694
4283,31099082072,Audrey,St-Germain,audreyst-germain@hotmail.ca,14503941540,2024-09-27,NaN,24.114.109.161,"Tue, 24 Sep 2024, 11:14 PM",14503941540
...,...,...,...,...,...,...,...,...,...,...
279,31054829988,MEZIERE Chantal,NaN,chantal.meziere@wanadoo.fr,590690643934,2024-09-24,NaN,112453,2023-11-12 11:07:27,590690643934
29568,31043178628,Karine,Morellon,NaN,+596696937550,2024-09-25,15/12/2021 09:40:27,54735,21/08/2020 08:36:05,596696937550
33396,31043195368,Karine,Morellon,NaN,596696937550,2024-09-24,18/02/2022 16:44:00,60019,08:36:05 21/08/2020,596696937550
39688,31043912459,Jeannine,Gourdine,jeannine.atam-kassigadou@orange.fr,+596696980678,2024-06-02,18/02/2022 16:44:55,61093,01:32:35 2022-02-12 + 2023-10-04T10:48:49-0500...,596696980678


In [70]:
del doublonsesx["cf_datecre_bitrix"]
del doublonsesx["cf_id_bitrix.1"]
del doublonsesx["simplified_mobile_number"]
doublonsesx

,internal_id,first_name,last_name,email,mobile_number,updated_at,cf_date_de_creation
41182,31056772427,Beatrice,Cervel,bea.lc85@hotmail.fr,0649758683,2024-05-06,18 août 2021
599,31054824717,Béa,tess,formation@institutadios.com,+0649758683,2024-09-26,2023-09-21T01:32:49+00:00
4292,31099082122,Lyne,Gastonguay,lynegastonguay@hotmail.com,14503043694,2024-09-27,"Tue, 24 Sep 2024, 02:38 AM"
901,31095016429,Lyne,Gastonguay,lynegastonguay@hotmail.com,+14503043694,2024-09-22,"Fri, 16 Aug 2024, 09:02 PM"
4283,31099082072,Audrey,St-Germain,audreyst-germain@hotmail.ca,14503941540,2024-09-27,"Tue, 24 Sep 2024, 11:14 PM"
...,...,...,...,...,...,...,...
279,31054829988,MEZIERE Chantal,NaN,chantal.meziere@wanadoo.fr,590690643934,2024-09-24,2023-11-12 11:07:27
29568,31043178628,Karine,Morellon,NaN,+596696937550,2024-09-25,21/08/2020 08:36:05
33396,31043195368,Karine,Morellon,NaN,596696937550,2024-09-24,08:36:05 21/08/2020
39688,31043912459,Jeannine,Gourdine,jeannine.atam-kassigadou@orange.fr,+596696980678,2024-06-02,01:32:35 2022-02-12 + 2023-10-04T10:48:49-0500...


In [71]:
doublonsesx.to_csv("Modif_Mobile_identique_mais_pas_meme_format.csv", index=False)
doublonsesx

,internal_id,first_name,last_name,email,mobile_number,updated_at,cf_date_de_creation
41182,31056772427,Beatrice,Cervel,bea.lc85@hotmail.fr,0649758683,2024-05-06,18 août 2021
599,31054824717,Béa,tess,formation@institutadios.com,+0649758683,2024-09-26,2023-09-21T01:32:49+00:00
4292,31099082122,Lyne,Gastonguay,lynegastonguay@hotmail.com,14503043694,2024-09-27,"Tue, 24 Sep 2024, 02:38 AM"
901,31095016429,Lyne,Gastonguay,lynegastonguay@hotmail.com,+14503043694,2024-09-22,"Fri, 16 Aug 2024, 09:02 PM"
4283,31099082072,Audrey,St-Germain,audreyst-germain@hotmail.ca,14503941540,2024-09-27,"Tue, 24 Sep 2024, 11:14 PM"
...,...,...,...,...,...,...,...
279,31054829988,MEZIERE Chantal,NaN,chantal.meziere@wanadoo.fr,590690643934,2024-09-24,2023-11-12 11:07:27
29568,31043178628,Karine,Morellon,NaN,+596696937550,2024-09-25,21/08/2020 08:36:05
33396,31043195368,Karine,Morellon,NaN,596696937550,2024-09-24,08:36:05 21/08/2020
39688,31043912459,Jeannine,Gourdine,jeannine.atam-kassigadou@orange.fr,+596696980678,2024-06-02,01:32:35 2022-02-12 + 2023-10-04T10:48:49-0500...


In [72]:
import gspread
import csv
from datetime import datetime

gc = gspread.service_account(filename="anomalieV2.json")

sh = gc.open_by_key('1cMEvZRMf8HWIeWIuZfGCojR4oMzPaJFGjnIEb4_4q8g')

worksheet = sh.worksheet("Anomalie 1-Mobile_identique_mais_pas_meme_format")

worksheet.clear()

with open("Modif_Mobile_identique_mais_pas_meme_format.csv", 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    rows = list(reader)

last_col_num = ord('A') + len(rows[0]) - 1
last_col_letter = chr(last_col_num) if last_col_num <= ord('Z') else 'Z'
cell_range = f'A2:{last_col_letter}{len(rows)+1}'

cell_list = worksheet.range(cell_range)

for cell, value in zip(cell_list, [cell for row in rows for cell in row]):
    cell.value = value

worksheet.update_cells(cell_list)

now = datetime.now()
date_string = now.strftime("%d/%m/%Y à %H:%M:%S")
worksheet.update_acell('J2', date_string)

worksheet = None
